In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
DEVICE='cuda'

In [19]:
import pandas as pd
from ast import literal_eval
import re
import json
import numpy as np
from tqdm.notebook import tqdm
import implicit

In [9]:
from scipy.sparse import dok_matrix

In [10]:
data = pd.read_excel('/storage1/ryazantsev/tenderhack/DataSet_EKB_200000.xlsx')

In [11]:
def trim_invalid_items(x):
    if x is np.nan:
        return '[]'
    else:
        rindex = x.rfind('}')
        return x[:rindex+1]+']'

In [12]:
data['Другая продукция в контрактах']=data['Другая продукция в контрактах'].map(trim_invalid_items)

In [14]:
data_other_notna = data[data['Другая продукция в контрактах'].notna()]['Другая продукция в контрактах']

In [15]:
json.loads(data['Другая продукция в контрактах'].iloc[5])[-2:]

[{'OtherSkuId': 34591891,
  'OtherSkuName': 'Ежедневник датированный 2021 год Attache Economy бумвинил A5 160 листов синий (147x206 мм)',
  'Quantity': 10.0},
 {'OtherSkuId': 35141961,
  'OtherSkuName': 'Ежедневник датированный 2022 год Attache Leader искусственная кожа А5+ 336 листов красный (146х206 мм)',
  'Quantity': 50.0}]

In [16]:
sku_numerator = {}

In [17]:
for i, index_ in enumerate(data['Идентификатор СТЕ'].values):
    sku_numerator[index_]=i

In [18]:
#prepare sparse matrix
cooccurence_matrix = dok_matrix((200000, 358000))
offset_index = 200000
invisible_sku = []

for sku_id_, ncontracts, other_sku in tqdm(data[['Идентификатор СТЕ', 'Кол-во заключенных контрактов', 'Другая продукция в контрактах']].itertuples(index=False)):
    other_sku = json.loads(other_sku)
    i = sku_numerator[sku_id_]
    for other_sku_ in other_sku:
        other_sku_id = other_sku_['OtherSkuId']
        if ncontracts>=1:
            quantity = other_sku_['Quantity']/ncontracts
            j = sku_numerator.get(other_sku_id)
            #if j and quantity>0.3:
            if j:
                #cooccurence_matrix[i,j] = quantity
                cooccurence_matrix[i,j] = min(quantity, 0.5)/0.5
            else:
                sku_numerator[other_sku_id] = offset_index
                invisible_sku.append(other_sku_)
                j = offset_index
                cooccurence_matrix[i,j] = min(quantity, 0.5)/0.5
                offset_index+=1

In [20]:
#Train model als
model = implicit.als.AlternatingLeastSquares(factors=25, regularization=0.1, iterations=20)
#alpha_val = 40
data_conf = (cooccurence_matrix).astype('double')
model.fit(data_conf)

In [24]:
def show_recommendations(index):
    recommended = model.similar_items(index,10)
    print(recommended)
    recommended_ind = list(map(lambda x: x[0], recommended))
    #return recommended_ind
    return data.iloc[recommended_ind]

In [28]:
show_recommendations(105)

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена
105,18077734,Блокнот Проф-пресс Городской стиль А5 80 листо...,Блокнот,Блокнот Проф-пресс Городской стиль А5 80 листо...,"[{""Name"":""Вид деловой бумажной продукции"",""Id""...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",6.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",РОССИЯ,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",8,66924.0,01.15.02.01,Городской стиль,"[{""Cost"":32.24000},{""Cost"":40.91000}]"
64201,34025096,Тетрадь общая Проф-пресс Панорамы А5 48 листов...,Тетрадь ученическая общая,NaN,"[{""Name"":""Тип крепления"",""Id"":-618845869,""Valu...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",3.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",РОССИЯ,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",3,1100146.0,01.15.02.06.02,1159171,"[{""Cost"":25.73000},{""Cost"":27.10000}]"
9281,1232842,Планшет BANTEX 4210-01 A4 синий с верх.створко...,Папки пластиковые,NaN,"[{""Name"":""Формат"",""Id"":291075708,""Value"":""А4""}...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",5.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",РОССИЯ,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",3,12114414.0,01.15.03.17.05,NaN,"[{""Cost"":296.00000}]"
4349,1232757,"Папка-регистратор BANTEX ECONOMY, 1446-09,б/ме...",Папки картонные,NaN,"[{""Name"":""Материал внешнего покрытия"",""Id"":368...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",8.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",РОССИЯ,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",8,12108439.0,01.15.03.16,"BANTEX ECONOMY, 1446-09,б/мет.уг,70мм,красн","[{""Cost"":283.00000}]"
91999,19427217,"Лезвия для канцелярских ножей Berlingo, 18мм, ...",Резаки канцелярские,NaN,"[{""Name"":""Запасное лезвие для канцелярского но...","[{""Name"":""Москва""},{""Name"":""Московская""}]",6.0,"[{""SupplierId"":1105608,""Name"":""Общество с огра...",КИТАЙ,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",7,12114422.0,01.15.05.16.02,Лезвия для канцелярских ножей Berlingo,NaN
92664,33703671,"Скрепки 28 мм, Berlingo, 100шт., золотистые, к...",Скрепки канцелярские,NaN,"[{""Name"":""Вид зажима для бумаг"",""Id"":-63195841...","[{""Name"":""Москва""},{""Name"":""Московская""}]",8.0,"[{""SupplierId"":1259802,""Name"":""ОБЩЕСТВО С ОГРА...",NaN,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",8,12114410.0,01.15.05.04.06.01,28 мм,"[{""Cost"":48.00000}]"
7237,1249260,"Корректирующая ручка 8мл KORES Preсiso, шарико...",Средства корректирующие канцелярские,NaN,"[{""Name"":""Страна происхождения"",""Id"":299363107...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",7.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",ЧЕШСКАЯ РЕСПУБЛИКА,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",2,12118572.0,01.15.05.04.03.01,NaN,"[{""Cost"":186.00000}]"
22697,1227102,"Коврик на стол Комус 47,5х66см прозрачный мато...",Коврики канцелярские,NaN,"[{""Name"":""Страна производитель"",""Id"":291010080...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",9.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",NaN,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",6,12108457.0,01.15.05.13,NaN,"[{""Cost"":502.20000},{""Cost"":806.07000}]"
36581,1397416,Скрепочница магнитная Attache Selection вертик...,Скрепочницы пластиковые,Скрепочница магнитная Attache Selection вертик...,"[{""Name"":""Цвет"",""Id"":325475640,""Value"":""черный...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",4.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",КИТАЙ,"[{""OtherSkuId"":1188993,""OtherSkuName"":""Berling...",6,12118602.0,01.15.05.01.05.01.02,Скрепочница магнитная вертикальная с чер

In [ ]:
#Dumps weights als
np.save('weights/item_factors.npy',item_factors)
np.save('weights/item_norms.npy',item_norms)